# Smart-G 3D demo notebook

This is an interactive document allowing to run Smart-G in 3D with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2:w
from __future__ import absolute_import, division, print_function
import os, sys
#sys.path.insert(0, os.path.dirname(os.getcwd()))
sys.path.insert(0, '/home/did/RTC/SMART-G/')
from smartg.albedo import Albedo_speclib, Albedo_cst, Albedo_spectrum
from smartg.smartg import Smartg, Sensor, type_Cell,type_Profile, type_Sensor
from smartg.smartg import RoughSurface, LambSurface, FlatSurface, Environment
from smartg.atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1, read_phase
from smartg.water import IOP_1, IOP, IOP_profile
from smartg.geometry import BBox, Point, Vector, CommonFace, CommonVertices
from smartg.reptran import REPTRAN, reduce_reptran
from smartg.tools.tools import SpherIrr, Irr, reduce_Irr
from luts.luts import LUT, MLUT, Idx, merge, read_mlut
from smartg.tools.smartg_view import compare, plot_polar, spectrum , mdesc 
from smartg.tools.smartg_view import spectrum_view,transect_view,profile_view,phase_view,smartg_view,input_view
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")
from scipy.interpolate import interp1d

def read_3dmcpol_a5_pp():
    
    f_iprt = '../smartg/validation/IPRT/iprt_case_a5pp_3dmcpol.dat'
    th0    = 50.
    mu0    = np.cos(np.radians(th0))
    thv    = np.arange(0.0, 81.0, 1.0)
    phi    = np.array([0.0, 180.0])

    mSvRG = MLUT()
    mSvRG.add_axis('Azimuth angles', phi)
    mSvRG.add_axis('Zenith angles',  thv)
    
    f=open(f_iprt)
    for k, lev2 in enumerate(['down (0+)','up (TOA)']):
        I = np.full( (len(phi), len(thv) ), np.nan)
        Q = np.full( (len(phi), len(thv) ), np.nan)
        U = np.full( (len(phi), len(thv) ), np.nan)
        V = np.full( (len(phi), len(thv) ), np.nan)
        Is = np.full( (len(phi), len(thv) ), np.nan)
        Qs = np.full( (len(phi), len(thv) ), np.nan)
        Us = np.full( (len(phi), len(thv) ), np.nan)
        Vs = np.full( (len(phi), len(thv) ), np.nan)
        for iphi in range(len(phi)): 
            for imu in range(len(thv)):
                tmp = f.readline().split()
                if (k==0):
                    I[iphi,imu] = float(tmp[6])   * np.pi/mu0
                    Q[iphi,imu] = float(tmp[7])   * np.pi/mu0
                    U[iphi,imu] = float(tmp[8])   * np.pi/mu0
                    V[iphi,imu] = float(tmp[9])   * np.pi/mu0
                    Is[iphi,imu] = float(tmp[10]) * np.pi/mu0
                    Qs[iphi,imu] = float(tmp[11]) * np.pi/mu0
                    Us[iphi,imu] = float(tmp[12]) * np.pi/mu0
                    Vs[iphi,imu] = float(tmp[13]) * np.pi/mu0
                else:
                    I[iphi,-imu-1] = float(tmp[6])   * np.pi/mu0
                    Q[iphi,-imu-1] = float(tmp[7])   * np.pi/mu0
                    U[iphi,-imu-1] = float(tmp[8])   * np.pi/mu0
                    V[iphi,-imu-1] = float(tmp[9])   * np.pi/mu0
                    Is[iphi,-imu-1] = float(tmp[10]) * np.pi/mu0
                    Qs[iphi,-imu-1] = float(tmp[11]) * np.pi/mu0
                    Us[iphi,-imu-1] = float(tmp[12]) * np.pi/mu0
                    Vs[iphi,-imu-1] = float(tmp[13]) * np.pi/mu0

        mSvRG.add_dataset('I_'+lev2, I, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('Q_'+lev2, Q, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('U_'+lev2, U, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('V_'+lev2, V, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('N_'+lev2, np.zeros_like(I),   ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('I_stdev_'+lev2, Is, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('Q_stdev_'+lev2, Qs, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('U_stdev_'+lev2, Us, ["Azimuth angles","Zenith angles"])
        mSvRG.add_dataset('V_stdev_'+lev2, Vs, ["Azimuth angles","Zenith angles"])

    return mSvRG 

def read_3dmcpol_c2_pp(iset=1, icase=4):
    
    f_iprt = '../smartg/validation/IPRT/iprt_case_C2_3DMCPOL.dat'
    th0    = 40.
    mu0    = np.cos(np.radians(th0))
    thv    = np.array([0.])
    phi    = np.array([0.])
    si     = np.arange(4900)

    mSvRG = MLUT()
    mSvRG.add_axis('Zenith angles',  thv)
    mSvRG.add_axis('sensor index' ,  si)
    
    f=open(f_iprt)
    lev2 = 'up (TOA)'

    I  = np.full( (len(si), len(thv) ), np.nan)
    Q  = np.full( (len(si), len(thv) ), np.nan)
    U  = np.full( (len(si), len(thv) ), np.nan)
    V  = np.full( (len(si), len(thv) ), np.nan)
    Is = np.full( (len(si), len(thv) ), np.nan)
    Qs = np.full( (len(si), len(thv) ), np.nan)
    Us = np.full( (len(si), len(thv) ), np.nan)
    Vs = np.full( (len(si), len(thv) ), np.nan)
    
    imu=0

    for k in range(3 + (4900*9*iset) + (4900*icase)):
        f.readline()
    for i in range(4900):
        tmp = f.readline().split()
        I[i,imu]  = float(tmp[7])  * np.pi/mu0
        Q[i,imu]  = float(tmp[8])  * np.pi/mu0
        U[i,imu]  = float(tmp[9])  * np.pi/mu0
        V[i,imu]  = float(tmp[10])  * np.pi/mu0
        Is[i,imu] = float(tmp[11]) * np.pi/mu0
        Qs[i,imu] = float(tmp[12]) * np.pi/mu0
        Us[i,imu] = float(tmp[13]) * np.pi/mu0
        Vs[i,imu] = float(tmp[14]) * np.pi/mu0
        
    mSvRG.add_dataset('I_'+lev2, I, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('Q_'+lev2, Q, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('U_'+lev2, U, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('V_'+lev2, V, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('N_'+lev2, np.zeros_like(I),   ["sensor index","Zenith angles"])
    mSvRG.add_dataset('I_stdev_'+lev2, Is, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('Q_stdev_'+lev2, Qs, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('U_stdev_'+lev2, Us, ["sensor index","Zenith angles"])
    mSvRG.add_dataset('V_stdev_'+lev2, Vs, ["sensor index","Zenith angles"])

    return mSvRG 


def Get_3Dcells_indices(NX, NY, NZ):
    Ncell = NX*NY*NZ
    # from cell number to x,y and z indices
    return np.unravel_index(np.arange(Ncell, dtype=int32), (NX, NY, NZ), order='C')


def Get_3Dcells_neighbours(NX, NY, NZ, BOUNDARY_ABS=-5, periodic=False,
                          BOUNDARY_BOA=-2, BOUNDARY_TOA=-1):
    '''
    Computes the 3D neighbouring cells indices, one for each of the 6 cube faces
    
    Keyword:
        - periodic : the neighbours are horizontally periodic, otherwise it is an absorbing boundary
    '''
    idx, idy, idz  = Get_3Dcells_indices(NX, NY, NZ)
    # indices of neighbouring cells in rectangular grid
    neigh_idx      = np.vstack((idx+1, idx-1, idx  , idx  , idx  , idx  )) # by convention POSITIVE first
    neigh_idy      = np.vstack((idy  , idy  , idy+1, idy-1, idy  , idy  ))
    neigh_idz      = np.vstack((idz  , idz  , idz  , idz  , idz+1, idz-1))
    
    if periodic: 
        neigh = np.ravel_multi_index((neigh_idx, neigh_idy, neigh_idz), 
                                       dims=(NX, NY, NZ), mode=('wrap','wrap','clip'))
    else:
        neigh = np.ravel_multi_index((neigh_idx, neigh_idy, neigh_idz), 
                                       dims=(NX, NY, NZ), mode=('clip','clip','clip'))
    ## boundaries neighbouring
    # with 'clip' mode, if outside the domain then the neighbour index is the same as the cell index
    neigh[np.equal(neigh , np.arange(NX*NY*NZ, dtype=int32))] = BOUNDARY_ABS 
    # by definition -Z neighbour at the domain boundary is BOA
    neigh[5, np.where(neigh[5,:]==BOUNDARY_ABS)] = BOUNDARY_BOA 
    # by definition +Z neighbour at the domain boundary is TOA
    neigh[4, np.where(neigh[4,:]==BOUNDARY_ABS)] = BOUNDARY_TOA
    # by convention +Z neighbour at the domain boundary -1 is also TOA
    neigh[4, np.where(neigh[4,:]%NZ==        0)] = BOUNDARY_TOA 
    
    return neigh.astype(int32)


def Get_3Dcells(Nx=1, Ny=1, Nz=50, Dx=1., Dy=1., Dz=1.,x=None, y=None, z=None, periodic=False,
                HORIZ_EXTENT_LENGTH=0, SAT_ALTITUDE=1e3):
    '''
    return the cells geometrical properties for use in 3D atmospheric profile object

    Keywords:
        - Nx, Ny and Nz are the number of cells in each dimension
        - Dx, Dy and Dz are the cells dimensions in km (default 1.)
        - x(Nx), y(Ny), z(Nz), coordinates can be provided instead, it erase Ni and Di
        - HORIZ_EXTENT_LENGTH: in km, is not 0, then one cell before and one after in X and Y 
            are added with a specific length of HORIZ_EXTENT_LENGTH. it results in a total
            number of cells being NX = Nx+2 and NY = Ny+2
        - SAT_ALTITUDE: Max altitude in km for sensors location within the 3D grid
        - periodic : the neighbours are horizontally periodic, otherwise it is an absorbing boundary
    '''
    # CELLS INDEXING
    DN             = 2 if HORIZ_EXTENT_LENGTH !=0 else 0
    sl             = slice(DN//2, -DN//2) if DN==2 else slice(None, None)
        
    if x is None:
        NX   = Nx + DN  # Number of cells in x
        # cells boundaries coordinates 
        # horizontal 
        Hx   = Nx*Dx/2.  # x central domain half length (km)
        x    = np.zeros((NX+1), dtype=np.float32)
        x[0] = -(Hx + HORIZ_EXTENT_LENGTH)
        x[-1]= (Hx  + HORIZ_EXTENT_LENGTH)
        x[sl]= np.linspace(-Hx, Hx, num=Nx+1)
    else:
        NX   = x.size-1
        
    if y is None:
        NY   = Ny + DN  # Number of cells in x
        # cells boundaries coordinates 
        # horizontal 
        Hy   = Ny*Dy/2.  # y central domain half length (km)
        y    = np.zeros((NY+1), dtype=np.float32)
        y[0] = -(Hy + HORIZ_EXTENT_LENGTH)
        y[-1]= (Hy  + HORIZ_EXTENT_LENGTH)
        y[sl]= np.linspace(-Hy, Hy, num=Ny+1)
    else:
        NY   = y.size-1
        
    # vertical boundaries
    if z is None :
        # we add a empty very thin cell above TOA (just for interfacing purposes)
        NZ    = Nz + 1
        z     = np.zeros((NZ+1), dtype=np.float32)
        z[:-1]= np.linspace(0, Nz*Dz, num=NZ)
        z[-1] = SAT_ALTITUDE # above TOA , sensor max level
    else:
        NZ    = z.size-1
    
    # from cell number to x,y and z indices
    idx, idy, idz  = Get_3Dcells_indices(NX, NY, NZ)
    # indices of neighbouring cells in rectangular grid
    neigh          = Get_3Dcells_neighbours(NX, NY, NZ, periodic=periodic)
    # Bounding boxes, lower left and upper right corners
    pmin           = np.zeros((3, NX*NY*NZ), dtype=np.float32)
    pmax           = np.zeros_like(pmin)
    pmin[0,:]      = x[idx]
    pmax[0,:]      = x[idx+1]
    pmin[1,:]      = y[idy]
    pmax[1,:]      = y[idy+1]
    pmin[2,:]      = z[idz] # ! from bottom to top
    pmax[2,:]      = z[idz+1]

    return (idx,idy,idz), (NX,NY,NZ), (x,y,z), neigh, pmin, pmax


def locate_3D_cells(pro3D, xs, ys, zs):
    cond = np.greater_equal(pro3D['pmax_atm'].data[0,:][None,:], xs[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[0,:][None,:], xs[:,None]) &\
        np.greater_equal(   pro3D['pmax_atm'].data[1,:][None,:], ys[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[1,:][None,:], ys[:,None]) &\
        np.greater_equal(   pro3D['pmax_atm'].data[2,:][None,:], zs[:,None]) &\
        np.less(            pro3D['pmin_atm'].data[2,:][None,:], zs[:,None])
    
    return np.where(cond)[1]



def locate_3Dregular_cells(xgrid,ygrid,zgrid,x,y,z):
    
    return  np.ravel_multi_index(( \
            np.floor(interp1d(xgrid, np.arange(len(xgrid)))(x)).astype(int),
            np.floor(interp1d(ygrid, np.arange(len(ygrid)))(y)).astype(int),
            np.floor(interp1d(zgrid, np.arange(len(zgrid)))(z)).astype(int)),
             dims = (len(xgrid)-1, len(ygrid)-1, len(zgrid)-1))


def od2k(prof, dataset, axis=1, zreverse=False):
    ot = diff1(prof[dataset].data.astype(np.float32), axis=axis)
    dz = diff1(prof.axis('z_atm')).astype(np.float32)
    k  = abs(ot/dz)
    k[np.isnan(k)] = 0
    sl = slice(None,None,-1 if zreverse else 1)
    
    return k[:,sl]

In [ ]:
# Kernel 2
from smartg.tools.modified_environ import modified_environ
import pycuda.driver as drv
import pycuda.tools

env_modif = {'CUDA_DEVICE': str(0)}
with modified_environ(**env_modif):
    import pycuda.autoinit
print(pycuda.autoinit.device.name())
from pycuda.compiler import SourceModule
from pycuda.gpuarray import to_gpu, zeros as gpuzeros
src_device = '/home/did/RTC/SMART-G/smartg/kernel2.cu'
source=open(src_device).read()
mod = SourceModule(source, nvcc='nvcc', no_extern_c=True)
reduce_absorption_gpu2 = mod.get_function("reduce_absorption_gpu2")
reduce_absorption_gpu  = mod.get_function("reduce_absorption_gpu")

# Introduction

## IPRT A5 case

In [ ]:
# Read 3DMCPOL from IPRT website
mcpol_pp=read_3dmcpol_a5_pp()

# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/watercloud_IPRT_A5.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = cld_phase['wavelen'].data*1e3
wl=[350.]
l=[]
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p=cld_phase['phase'][:,:,0,:ntheta].ravel()
theta=cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu= np.cos(np.radians(theta))
Norm = np.trapz(p,-mu)
data = np.array(l) 
print ('phase function norm: ', Norm)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[None, None, :, ::-1], names=['wav_phase', 'z_phase', 'stk', 'theta_atm'],
          axes=[wl, np.array([0.]), None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

# Layering and angles
th0  = 50.
COT  = 5. # at 800 nm
cld  = CloudOPAC('wc.sol', 3., 0.5, 10., COT, wl, phase=pha_cld)
NL   = 1
grid = np.linspace(100, 0., num=NL+1)
atm  = AtmAFGL('afglt', O3=0., comp=[cld], tauR=1e-20, NO2=False, grid=grid)
le   = {'phi_deg':np.linspace(0.,180., num=2), 'th_deg':np.linspace(0., 80., num=81)}

### Principal plane

In [ ]:
%%time
ma5_1e8 = Smartg(double=True, alt_pp=False).run(wl=800., atm=atm, le=le, 
                                            NBPHOTONS=1e5, OUTPUT_LAYERS=3, THVDEG=th0,
                                           stdev=True)

In [ ]:
# 3DMCPOL 1e8 photons 21 hours 25 min for one field (TOA)
# 
T = (21*3600+ 25*60)*2.
print ('3DMCPOL: %.1f'%T)
print('%.2f'%(float(ma5_1e8.attrs['kernel time (s)'])))

In [ ]:
_=compare(ma5_1e8, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.01, 0.001, 0.001, 0.5], ermax=[1,5,20,5])

In [ ]:
_=compare(ma5_1e8, mcpol_pp,same_azimuth_convention=True, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.05, 0.001, 10], vmin=[-1., -0.05, -0.001, 0], field='down (0+)', logI=True,
        emax=[0.5, 0.001, 0.001, 0.5], ermax=[5,5,20,5])

In [ ]:
%%time
ma5_1e6 = Smartg(double=True, alt_pp=False).run(wl=800., atm=atm, le=le, 
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=3, THVDEG=th0,
                                           stdev=True)

In [ ]:
_=compare(ma5_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.1, 0.01, 0.01, 1], ermax=[10,10,10,10])

In [ ]:
%%time
ma5_alt_1e6 = Smartg(double=True, alt_pp=True).run(wl=800., atm=atm, le=le,
                                            NBPHOTONS=1e6, OUTPUT_LAYERS=3, THVDEG=th0,
                                            stdev=True)

In [ ]:
_=compare(ma5_alt_1e6, mcpol_pp,same_azimuth_convention=False, azimuth=[0.0, 0.0], errb=True,
        vmax=[1.5, 0.1, 0.01, 50], vmin=[0, -0.1, -0.01, 0], 
        emax=[0.1, 0.01, 0.01, 1], ermax=[10,10,10,10])

# 3D

## Bounding Box

![Image of Bounding Box](https://static.packt-cdn.com/products/9781787123663/graphics/B05887_7_5.jpg)

## Cube Mapping

![Image of Cube Mapping](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Cube_map.svg/599px-Cube_map.svg.png)

## 3D profiles from 1D profiles an Bounding Boxes Grid

**1D profiles with "OPT3D" option**

In [ ]:
wl = [510.]
wl = np.linspace(420., 500., num=21)
znew = np.array([100.,80.,60.,40.,20.,15.,10.,9.,8.,7.,6.,5.,4.,3.,2.,1.,0.])
NL   = len(znew)
print('NLayer:', NL)
COT = 5. # at 800 nm
cld  = CloudOPAC('wc.sol', 3., 9., 10., COT, wl[0], phase=pha_cld)
cld0 = CloudOPAC('wc.sol', 3., 1., 2., 2.,  wl[0], phase=pha_cld)
pfwav = [wl[0]]
pro_clr3D = AtmAFGL('afglt', grid=znew, OPT3D=True,  pfwav= pfwav).calc(wl, phase=True)
pro_cld3D = AtmAFGL('afglt', grid=znew, comp=[cld0,cld],  OPT3D=True,  pfwav=pfwav).calc(wl, phase=True)
pro_clr1D = AtmAFGL('afglt', grid=znew, OPT3D=False, pfwav= pfwav).calc(wl, phase=True)
pro_cld1D = AtmAFGL('afglt', grid=znew, comp=[cld0,cld] , OPT3D=False, pfwav= pfwav).calc(wl, phase=True)

pro_aer3D = AtmAFGL('afglt', grid=znew, comp=[AeroOPAC('urban', 5., 550.)],  OPT3D=True,  pfwav=pfwav).calc(wl, phase=True)

# Dev Image
pro_clr3D2= AtmAFGL('afglt', grid=znew, comp=[cld0],OPT3D=True,  O3=1000.,  
                    pfwav= pfwav).calc(wl, phase=True)

In [ ]:
# OPT3D=True allows vertical limits of Bounding Boxes to be 
# defined
pro_cld3D.describe()

**3D profile from 1D profiles : Horizontal Boundaries**

In [ ]:
# Simplest case : just one profile
HLONGX= 1e4
HLONGY= 1e4
H_boundaries = np.array([[-HLONGX, HLONGX, -HLONGY, HLONGY]])
p3D = pro3D_from_1D([pro_clr3D], H_boundaries=H_boundaries).describe()
p1D = pro_clr1D.describe()

In [ ]:
pcolormesh(p3D['neighbour_atm'].data, vmin=-5, vmax=None, cmap=cm.jet)
xlabel('BBox index')
ylabel('Face number')
title('Neighbouring box indices per face')
colorbar()
# -5 : absorbing boundary
# -2 : surface
# -1 : TOA

In [ ]:
S3D = Smartg(opt3D=True, alt_pp=True) # 3D

In [ ]:
le={'th_deg':np.linspace(0., 80., num=1), 'phi_deg':np.linspace(30., 210., num=1)}
surf=None

POSZ, POSX, POSY = 99.99, -0.1, -0.1 # TOA sensor
THDEG, PHDEG = 120., 0.   # Oblique view (VZA : 180-120 = 60)
IBOX = get_iboxes(p3D, poss=[Point(POSX, POSY, POSZ)]) # inittialize BBox id from position and 3D profile
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
NB=1e6

m3D  = S3D.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
               stdev=True, NF=1e4, progress=False)

In [ ]:
m3D['I_up (TOA)'].sub()[:,0,0].plot()

## Compilation options

In [ ]:
# Backward mode activated for 3D comparisons
S   = Smartg(back=True, double=True, device=0) # "Fast" 1D PP (~ Maximum cross section or Null ?)
S0  = Smartg(alt_pp=True, back=True, double=True) # Layers loop 1D PP
S3D = Smartg(opt3D=True, alt_pp=True, back=True, double=True) # 3D
Ssp = Smartg(pp=False, back=True, double=True) # Layers loop 1S Spherical

# ALIS version (for clear atmospheres)
S0A = Smartg(alt_pp=True, alis=True, back=True, double=True)
S3DA= Smartg(opt3D=True,  alt_pp=True, alis=True, back=True, double=True)
SspA= Smartg(pp=False,    alis=True, back=True, double=True)
surf  = None

### One solar direction, One pixel

#### Local estimate

In [ ]:
le={'th_deg':np.linspace(0., 80., num=1), 'phi_deg':np.linspace(30., 210., num=1)}

POSZ, POSX, POSY = 99.99, -0.1, -0.1 # TOA sensor
THDEG, PHDEG = 120., 0.   # Oblique view (VZA : 180-120 = 60)
IBOX = get_iboxes(p3D, poss=[Point(POSX, POSY, POSZ)]) # inittialize BBox id from position and 3D profile
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
NB=1e6
m    = S.run(  wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
             stdev=True, NF=1e4, progress=False)
m0   = S0.run( wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
              stdev=True, NF=1e4, progress=False)
m3D  = S3D.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
               stdev=True, NF=1e4, progress=False)

m0A =   S0A.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
              alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)
m3DA = S3DA.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
              alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)

RTER   = 6370.
sensor = Sensor(POSZ=POSZ+RTER, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
msp    = Ssp.run(wl=wl,   NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
                 stdev=True, NF=1e4, RTER=RTER, progress=False)
mspA   = SspA.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
                 RTER=RTER, alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)

print ('No ALIS, NB: %.0e'%NB)
for version, result in zip(['fast_1D_PP','1D_PP','1D_SP', '3D_PP'], [m,m0,msp,m3D]):
    print("{0:10s} I: {1:10.3e}+-{4:9.3e},Q: {2:10.3e}+-{5:9.3e},U: {3:10.3e}+-{6:9.3e}, k. time (s): {7:.3f}"\
                          .format(version, float(result['I_up (TOA)'].data[0]), 
                                  float(result['Q_up (TOA)'].data[0]), 
                                  float(result['U_up (TOA)'].data[0]),
                                  float(result['I_stdev_up (TOA)'].data[0]), 
                                  float(result['Q_stdev_up (TOA)'].data[0]), 
                                  float(result['U_stdev_up (TOA)'].data[0]),
                                  float(result.attrs['kernel time (s)'])))
print('=====')
print ('ALIS, NB: %.0e'%NB)
for version, result in zip(['1D_PP','1D_SP', '3D_PP'], [m0A,mspA,m3DA]):
    print("{0:10s} I: {1:10.3e}+-{4:9.3e},Q: {2:10.3e}+-{5:9.3e},U: {3:10.3e}+-{6:9.3e}, k. time (s): {7:.3f}"\
                          .format(version, float(result['I_up (TOA)'].data[0]), 
                                  float(result['Q_up (TOA)'].data[0]), 
                                  float(result['U_up (TOA)'].data[0]),
                                  float(result['I_stdev_up (TOA)'].data[0]), 
                                  float(result['Q_stdev_up (TOA)'].data[0]), 
                                  float(result['U_stdev_up (TOA)'].data[0]),
                                  float(result.attrs['kernel time (s)'])))
print ('-----------------------')


#### Cone sampling

In [ ]:
le=None

POSZ, POSX, POSY = 99.99, -0.1, -0.1 # TOA sensor
THDEG, PHDEG = 120., 0.   # Oblique view (VZA : 180-120 = 60)
IBOX = get_iboxes(p3D, poss=[Point(POSX, POSY, POSZ)]) # inittialize BBox id from position and 3D profile
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
NB=1e7
m    = S.run(  wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
             stdev=True, NF=1e4, progress=False)
m0   = S0.run( wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
              stdev=True, NF=1e4, progress=False)
m3D  = S3D.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
               stdev=True, NF=1e4, progress=False)

m0A =   S0A.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
               alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)
m3DA = S3DA.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
               alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)

RTER   = 6370.
sensor = Sensor(POSZ=POSZ+RTER, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
msp    = Ssp.run(wl=wl,   NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
                 stdev=True, NF=1e4, RTER=RTER, progress=False)
mspA   = SspA.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
                  RTER=RTER,
               alis_options={'nlow':-1}, stdev=True, NF=1e4, progress=False)

print ('No ALIS, NB: %.0e'%NB)
for version, result in zip(['fast_1D_PP','1D_PP','1D_SP', '3D_PP'], [m,m0,msp,m3D]):
    result=result.sub({'Azimuth angles':0, 'Zenith angles':0})
    print("{0:10s} I: {1:10.3e}+-{4:9.3e},Q: {2:10.3e}+-{5:9.3e},U: {3:10.3e}+-{6:9.3e}, k. time (s): {7:.3f}"\
                          .format(version, float(result['I_up (TOA)'].data[0]), 
                                  float(result['Q_up (TOA)'].data[0]), 
                                  float(result['U_up (TOA)'].data[0]),
                                  float(result['I_stdev_up (TOA)'].data[0]), 
                                  float(result['Q_stdev_up (TOA)'].data[0]), 
                                  float(result['U_stdev_up (TOA)'].data[0]),
                                  float(result.attrs['kernel time (s)'])))
print('=====')
print ('ALIS, NB: %.0e'%NB)
for version, result in zip(['1D_PP','1D_SP', '3D_PP'], [m0A,mspA,m3DA]):
    result=result.sub({'Azimuth angles':0, 'Zenith angles':0})
    print("{0:10s} I: {1:10.3e}+-{4:9.3e},Q: {2:10.3e}+-{5:9.3e},U: {3:10.3e}+-{6:9.3e}, k. time (s): {7:.3f}"\
                          .format(version, float(result['I_up (TOA)'].data[0]), 
                                  float(result['Q_up (TOA)'].data[0]), 
                                  float(result['U_up (TOA)'].data[0]),
                                  float(result['I_stdev_up (TOA)'].data[0]), 
                                  float(result['Q_stdev_up (TOA)'].data[0]), 
                                  float(result['U_stdev_up (TOA)'].data[0]),
                                  float(result.attrs['kernel time (s)'])))
print ('-----------------------')


### Several direction, One pixel

In [ ]:
le={'th_deg':np.linspace(0., 80., num=6), 'phi_deg':np.linspace(30., 210., num=2)}
NB=1e6
POSZ, POSX, POSY  = 99.99, -0.1, -0.1
THDEG, PHDEG = 120., 0.
IBOX = get_iboxes(p3D, poss=[Point(POSX, POSY, POSZ)])
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
m    = S.run(  wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, stdev=True, NF=1e4)
m0   = S0.run( wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, stdev=True, NF=1e4)
m3D  = S3D.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, stdev=True, NF=1e4)

m0A =   S0A.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, 
               alis_options={'nlow':-1}, stdev=True, NF=1e4)
m3DA = S3DA.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, 
               alis_options={'nlow':-1}, stdev=True, NF=1e4)

RTER   = 6370.
sensor = Sensor(POSZ=POSZ+RTER, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)
msp    = Ssp.run(wl=wl,   NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, stdev=True, NF=1e4, RTER=RTER)
mspA   = SspA.run(wl=wl,  NBPHOTONS=NB, atm=p1D, sensor=sensor, le=le, surf=surf, RTER=RTER,
               alis_options={'nlow':-1}, stdev=True, NF=1e4)

In [ ]:
QU=True
f=transect_view(m3D.sub({'wavelength':0}), ind=Idx([30]), color='r', fmt='--', QU=QU)
_=transect_view(msp.sub({'wavelength':0}), ind=Idx([30]), fig=f,    color='b', fmt=':', QU=QU)
_=transect_view(m0.sub({'wavelength':0}),  ind=Idx([30]), fig=f,    color='k', fmt='x', QU=QU)
_=transect_view(m.sub({'wavelength':0}),   ind=Idx([30]), vmin=None, vmax=None, fig=f, color='k', fmt='.', QU=QU)

In [ ]:
QU=True
f=transect_view(m3DA.sub({'wavelength':0}), ind=Idx([30]), color='r', fmt='--', QU=QU)
_=transect_view(mspA.sub({'wavelength':0}), ind=Idx([30]), fig=f,    color='b', fmt=':', QU=QU)
_=transect_view(m0A.sub({'wavelength':0}),  ind=Idx([30]), fig=f,    color='k', fmt='x', QU=QU)

In [ ]:
f=figure()
f.set_size_inches(12,3)
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m3DA['I_up (TOA)'].sub()[:,0,k]
    Q=m3DA['Q_up (TOA)'].sub()[:,0,k]
    U=m3DA['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='-'+col,vmin=0, label='3D '+'%.1f'%th)
    
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m0A['I_up (TOA)'].sub()[:,0,k]
    Q=m0A['Q_up (TOA)'].sub()[:,0,k]
    U=m0A['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='s'+col,vmin=0.0, vmax=0.25, label='1D')

legend(loc=3, ncol=4)

In [ ]:
ith=-1
norm=m3DA['N_up (TOA)'][0,0,ith]
plot(m3DA['cdist_up (TOA)'][:,0,ith]/norm, m3DA.axis('z_atm')[:-1],'-+', label='3D')
norm=m0A ['N_up (TOA)'][0,0,ith]
plot(m0A ['cdist_up (TOA)'][:,0,ith]/norm, m3DA.axis('z_atm')[0:-1],':', label='1D')
norm=mspA['N_up (TOA)'][0,0,ith]
plot(mspA['cdist_up (TOA)'][:,0,ith]/norm, m3DA.axis('z_atm')[0:-1],'--', label='1D sph.')
xlabel('Mean Distance in layer (km)')
ylabel('z (km)')
ylim([0,100])
legend()

In [ ]:
m3D.attrs['kernel time (s)'], m0.attrs['kernel time (s)'] , m.attrs['kernel time (s)'], msp.attrs['kernel time (s)']

In [ ]:
m3DA.attrs['kernel time (s)'], m0A.attrs['kernel time (s)'] , mspA.attrs['kernel time (s)']

## 3D vs 1D, two clear profile

In [ ]:
HLONGX= 1e4
HLONGY= 1e4
H_boundaries = np.array([[-HLONGX, 0., -HLONGY, HLONGY], [0., HLONGX, -HLONGY, HLONGY]])
p3D = pro3D_from_1D([pro_clr3D, pro_clr3D],
                   H_boundaries=H_boundaries).describe()

In [ ]:
p3D['i_atm'][:]

In [ ]:
pcolormesh(p3D['neighbour_atm'].data, vmin=-5, vmax=None, cmap=cm.jet)
colorbar()
p3D.axis('z_atm')

In [ ]:
POSZ, POSX, POSY = 99.99, -0.1, -0.1
THDEG, PHDEG = 120., 0.
IBOX = get_iboxes(p3D, poss=[Point(POSX, POSY, POSZ)])
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)

NB=1e6
m3D  = S3D.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, stdev=True, NF=1e4)

NB=1e6
m3DA = S3DA.run(wl=wl,  NBPHOTONS=NB, atm=p3D, sensor=sensor, le=le, surf=surf, NF=1e4,
               alis_options={'nlow':-1}, stdev=True)

In [ ]:
QU=True
f=transect_view(m3DA.sub({'wavelength':0}), ind=Idx([30]), color='r', fmt='--', QU=QU)
_=transect_view(mspA.sub({'wavelength':0}), ind=Idx([30]), fig=f,    color='b', fmt=':', QU=QU)
_=transect_view(m0A.sub({'wavelength':0}),  ind=Idx([30]), fig=f,    color='k', fmt='x', QU=QU)

In [ ]:
f=figure()
f.set_size_inches(12,3)
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m3DA['I_up (TOA)'].sub()[:,0,k]
    Q=m3DA['Q_up (TOA)'].sub()[:,0,k]
    U=m3DA['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='-'+col,vmin=0, label='3D '+'%.1f'%th)
    
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m0A['I_up (TOA)'].sub()[:,0,k]
    Q=m0A['Q_up (TOA)'].sub()[:,0,k]
    U=m0A['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='s'+col,vmin=0, vmax=0.25, label='1D')

legend(loc=3, ncol=4)

In [ ]:
ith=-1
norm=m3DA['N_up (TOA)'][0,0,ith]
plot(m3DA['cdist_up (TOA)'][:,0,ith]/norm, m3DA.axis('z_atm')[0:-1],'.b', label='3D')
plot((m3DA['cdist_up (TOA)'][1:NL,0,ith]+m3DA['cdist_up (TOA)'][NL:,0,ith])/norm, 
         m0A.axis('z_atm')[0:-1],'-b', label='3D (total)')
norm=m0A ['N_up (TOA)'][0,0,ith]
plot(m0A ['cdist_up (TOA)'][:,0,ith]/norm, m0A.axis('z_atm')[0:-1],':', label='1D')
xlabel('Mean Distance in layer (km)')
ylabel('z (km)')
ylim([0,100])
legend()

In [ ]:
m3D.attrs['kernel time (s)'], m0.attrs['kernel time (s)'] , m.attrs['kernel time (s)'], msp.attrs['kernel time (s)']

In [ ]:
m3DA.attrs['kernel time (s)'], m0A.attrs['kernel time (s)'] , mspA.attrs['kernel time (s)']

## 3D vs 1D, NxM clear profiles

In [ ]:
HLONGX = 1e4
HLONGY = 1e4
HX     = 3.
HY     = 3.
N  = 1
M  = 1
DX = 2*HX/N
DY = 2*HY/M
x  = np.zeros((N+2))
y  = np.zeros((M+2))
DXbound = (HLONGX-HX)
DYbound = (HLONGY-HY)
x[0]    = -HLONGX + DXbound/2
x[-1]   =  HLONGX - DXbound/2.
x[1:-1] =  np.linspace(-HX + DX/2., HX - DX/2., num=N)
y[0]    = -HLONGY + DYbound/2
y[-1]   =  HLONGY - DYbound/2.
y[1:-1] =  np.linspace(-HY + DY/2., HY - DY/2., num=M)
X0, Y0  =  np.meshgrid(x,y)
Lx=[DXbound]
Lx.extend([DX]*N)
Lx.extend([DXbound])
Ly=[DYbound]
Ly.extend([DY]*M)
Ly.extend([DYbound])
DX0, DY0=  np.meshgrid(Lx,Ly)
H = []
for xi,yi,dxi,dyi in zip(X0.ravel(),Y0.ravel(),DX0.ravel(),DY0.ravel()):
    H.append(list((xi-dxi/2, xi+dxi/2, yi-dyi/2, yi+dyi/2)))
H_boundaries = np.array(H).reshape(((N+2)*(M+2),4))

plist=[pro_clr3D]*4
plist.extend([pro_cld3D])
plist.extend([pro_clr3D]*4)
p3D_NxM = pro3D_from_1D(plist, H_boundaries=H_boundaries).describe()

In [ ]:
pcolormesh(p3D_NxM['neighbour_atm'].data, vmin=0, vmax=None, cmap=cm.jet)
colorbar()

In [ ]:
POSZ, POSX, POSY = 99.99, 0., 0.
THDEG, PHDEG = 120., 0.
IBOX = get_iboxes(p3D_NxM, poss=[Point(POSX, POSY, POSZ)])
sensor = Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)

NB=1e6
m3D_NxM  = S3D.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensor, le=le, surf=surf, stdev=True)

NB=1e6
m3DA_NxM = S3DA.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensor, le=le, surf=surf, 
               alis_options={'nlow':-1}, stdev=True)

In [ ]:
QU=True
f=transect_view(m3D_NxM.sub({'wavelength':0}), ind=Idx([30]), color='r', fmt='--', QU=QU)
_=transect_view(msp.sub({'wavelength':0}), ind=Idx([30]), fig=f,    color='b', fmt=':', QU=QU)
_=transect_view(m0.sub({'wavelength':0}),  ind=Idx([30]), fig=f,    color='k', fmt='x', QU=QU)
_=transect_view(m.sub({'wavelength':0}),   ind=Idx([30]), vmin=None, vmax=None, fig=f, color='k', fmt='.', QU=QU)

In [ ]:
QU=True
f=transect_view(m3DA_NxM.sub({'wavelength':0}), ind=Idx([30]), color='r', fmt='--', QU=QU)
_=transect_view(mspA.sub({'wavelength':0}), ind=Idx([30]), fig=f,    color='b', fmt=':', QU=QU)
_=transect_view(m0A.sub({'wavelength':0}),  ind=Idx([30]), fig=f,    color='k', fmt='x', QU=QU)

In [ ]:
f=figure()
f.set_size_inches(12,3)
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m3DA_NxM['I_up (TOA)'].sub()[:,0,k]
    Q=m3DA_NxM['Q_up (TOA)'].sub()[:,0,k]
    U=m3DA_NxM['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='-'+col,vmin=0, label='3D '+'%.1f'%th)
    
for k,(col,th) in enumerate(zip(['r','g','b','k','m','y'], np.rad2deg(le['th']))):
    I=m0A['I_up (TOA)'].sub()[:,0,k]
    Q=m0A['Q_up (TOA)'].sub()[:,0,k]
    U=m0A['U_up (TOA)'].sub()[:,0,k]

    LP = (Q*Q + U*U).apply(sqrt, mdesc('LP_up (TOA)'))
    P = (LP/I*100).apply(abs,'DoLP')
    LP.plot(fmt='s'+col,vmin=0, vmax=0.25, label='1D')

legend(loc=3, ncol=4)

In [ ]:
ith=-1
norm=m3DA_NxM['N_up (TOA)'][0,0,ith]
plot(m3DA_NxM['cdist_up (TOA)'][1:,0,ith]/norm, m3DA_NxM.axis('z_atm')[1:-1],'.b', label='3D')
norm=m0A ['N_up (TOA)'][0,0,ith]
plot(m0A ['cdist_up (TOA)'][:,0,ith]/norm, m0A.axis('z_atm')[:-1],':', label='1D')
xlabel('Mean Distance in cell (km)')
ylabel('z (km)')
ylim([0,100])
xlim(0,300)
legend()

In [ ]:
norm=m3DA_NxM ['N_up (TOA)'][0,0,0]
dz = abs(np.diff(m3DA_NxM.axis('z_atm')))
plot(m3DA_NxM['cdist_up (TOA)'][:,0,0]/norm/dz, m3DA_NxM.axis('z_atm')[:-1], '.b', label='3D')
norm=m0A ['N_up (TOA)'][0,0,0]
dz = abs(np.diff(m0A.axis('z_atm')))
plot(m0A['cdist_up (TOA)'][:,0,0]/norm/dz, m0A.axis('z_atm')[:-1], ':', label='1D')
xlabel('Mean Air mass factor in cell')
ylabel('z (km)')
ylim([0,100])
xlim(0,50)
legend()

In [ ]:
m3D_NxM.attrs['kernel time (s)'], m0.attrs['kernel time (s)'] , m.attrs['kernel time (s)'], msp.attrs['kernel time (s)']

In [ ]:
m3DA_NxM.attrs['kernel time (s)'], m0A.attrs['kernel time (s)'] , mspA.attrs['kernel time (s)']

# Advanced: IPRT Case Cube Cloud

In [ ]:
#S3DA = Smartg(opt3D=True, alt_pp=True, back=True, double=True, alis=True)
S3D  = Smartg(opt3D=True, alt_pp=True, back=True, double=True, alis=False)

In [ ]:
# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/watercloud_IPRT_A5.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = cld_phase['wavelen'].data*1e3
wl=[350.]
l=[]
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p=cld_phase['phase'][:,:,0,:ntheta].ravel()
theta=cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu= np.cos(np.radians(theta))
Norm = np.trapz(p,-mu)
data = np.array(l) 
print ('phase function norm: ', Norm)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[None, None, :, ::-1], names=['wav_phase', 'z_phase', 'stk', 'theta_atm'],
          axes=[wl, np.array([0.]), None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

In [ ]:
# Layering and angles
'''le   = {'th_deg':np.array([20,20,20,20,40,40,40,40,40], dtype=float), 
        'phi_deg': np.array([0,60,120,180,0,0,60,120,180], dtype=float), 'zip':True}'''
wl = [800.]

znew = [5., 3., 2., 0.]
#znew = [50., 3., 2.5, 2., 1., 0.]
#znew = np.linspace(5, 0, num=25)
NL   = len(znew)
print('NLayer:', NL)
COT = 10. # at 800 nm
cld  = CloudOPAC('wc.sol', 10., 2., 3., COT, wl, phase=pha_cld)
pfwav = [wl[0]]

if False:
    ray  = np.atleast_2d([0., 0., 0., 0.])
    #ray  = np.atleast_2d([0.]*25)
else:
    ray  = np.atleast_2d([0., 0.2, 0.1, 0.2])
    
pro_clr3D = AtmAFGL('afglt', O3=0., NO2=False, grid=znew, OPT3D=True, prof_ray=ray,
                    pfwav= pfwav).calc(wl, phase=True)
pro_cld3D = AtmAFGL('afglt', O3=0., NO2=False, grid=znew, comp=[cld], OPT3D=True,  
                    prof_ray=ray, pfwav=pfwav).calc(wl, phase=True)

In [ ]:
HLONGX = 1e4
HLONGY = 1e4
HX     = 0.5
HY     = 0.5
N  = 1
M  = 1
DX = 2*HX/N
DY = 2*HY/M
x  = np.zeros((N+2))
y  = np.zeros((M+2))
DXbound = (HLONGX-HX)
DYbound = (HLONGY-HY)
x[0]    = -HLONGX + DXbound/2
x[-1]   =  HLONGX - DXbound/2.
x[1:-1] =  np.linspace(-HX + DX/2., HX - DX/2., num=N)
y[0]    = -HLONGY + DYbound/2
y[-1]   =  HLONGY - DYbound/2.
y[1:-1] =  np.linspace(-HY + DY/2., HY - DY/2., num=M)
X0, Y0  =  np.meshgrid(x,y)
Lx=[DXbound]
Lx.extend([DX]*N)
Lx.extend([DXbound])
Ly=[DYbound]
Ly.extend([DY]*M)
Ly.extend([DYbound])
DX0, DY0=  np.meshgrid(Lx,Ly)
H = []
for xi,yi,dxi,dyi in zip(X0.ravel(),Y0.ravel(),DX0.ravel(),DY0.ravel()):
    H.append(list((xi-dxi/2, xi+dxi/2, yi-dyi/2, yi+dyi/2)))
H_boundaries = np.array(H).reshape(((N+2)*(M+2),4))

plist=[pro_clr3D]*4
plist.extend([pro_cld3D])
plist.extend([pro_clr3D]*4)
p3D_NxM = pro3D_from_1D(plist, H_boundaries=H_boundaries).describe()

In [ ]:
p3D_NxM['OD_r'][0,:]

In [ ]:
pcolormesh(p3D_NxM['neighbour_atm'].data, vmin=0, vmax=None, cmap=cm.jet)
colorbar()

In [ ]:
%%time
POSZ = 4.9999
THDEG, PHDEG = 180., 0.
#pixels
NX = 70
NY = 70
NP = NX*NY
VX = 3.5
VY = 3.5
Ix = np.linspace(-VX, VX, num=NX) 
Iy = np.linspace(-VY, VY, num=NY) 

px, py   = np.meshgrid(Ix, Iy)
poss = []
for POSX,POSY in zip(px.ravel(), py.ravel()):
    poss.append(Point(POSX, POSY, POSZ))
iboxes = get_iboxes(p3D_NxM, poss)

sensors=[]
for pos,IBOX in zip(poss,iboxes):
    sensors.append(Sensor(POSZ=pos.z, POSX=pos.x, POSY=pos.y, FOV=90., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX))
    

IBOX0   = get_iboxes(p3D_NxM, [Point(0, 0, POSZ)])
sensor0 = Sensor(POSZ=POSZ, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX0)   

In [ ]:
%%time
POSZ = 4.9999
THDEG, PHDEG = 180., 0.
#pixels
NX = 70
NY = 70
NP = NX*NY
VX = 3.5
VY = 3.5
Ix = np.linspace(-VX, VX, num=NX) 
Iy = np.linspace(-VY, VY, num=NY) 

px, py   = np.meshgrid(Ix, Iy)
sensors=[]
for POSX,POSY in zip(px.ravel(), py.ravel()):
    IBOX = get_iboxes(p3D_NxM, poss=[Point(POSX, POSY, POSZ)])
    sensors.append(Sensor(POSZ=POSZ, POSX=POSX, POSY=POSY, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX)) 

In [ ]:
%%time
# case 5
le   = {'th_deg':np.array([40], dtype=float), 
        'phi_deg': np.array([180], dtype=float), 'zip':True}
surf=LambSurface(ALB=0.2)
NB=1e5*NP
#NB=1e7
m3D_NxM  = S3D.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensors,NBLOOP=NB/10, XGRID=624, XBLOCK=256,
                   reflectance=True, le=le, surf=surf, stdev=True)
print(m3D_NxM.attrs['kernel time (s)'])

In [ ]:
smartg_c2 = m3D_NxM.dropaxis('Zenith angles').dropaxis('Azimuth angles').dropaxis('wavelength')
mcpol_c2  = read_3dmcpol_c2_pp(iset=1).dropaxis('Zenith angles')

In [ ]:
for k in range(1):
    fig,ax=subplots(2,2)
    fig.set_size_inches((14,10))
    I = smartg_c2['I_up (TOA)'][:].reshape((NX,NY))
    Q = smartg_c2['Q_up (TOA)'][:].reshape((NX,NY))
    U = smartg_c2['U_up (TOA)'][:].reshape((NX,NY))
    V = smartg_c2['V_up (TOA)'][:].reshape((NX,NY))
    P1=ax[0,0].imshow(I, vmin=0.0, vmax=0.6, cmap=cm.jet)
    colorbar(P1, ax=ax[0,0], shrink=0.8)
    P2=ax[0,1].imshow(Q, vmin=-0.1, vmax=0.1, cmap=cm.RdBu_r)
    colorbar(P2, ax=ax[0,1], shrink=0.8)
    P3=ax[1,0].imshow(-U, vmin=-0.0005, vmax=0.0005, cmap=cm.RdBu_r)
    colorbar(P3, ax=ax[1,0], shrink=0.8)
    P4=ax[1,1].imshow(V, vmin=-0.0001, vmax=0.0001, cmap=cm.RdBu_r)
    colorbar(P4, ax=ax[1,1], shrink=0.8)
    #P4=ax[1,1].imshow(np.sqrt(Q**2+U**2)/I*100, vmin=0, vmax=40, cmap=cm.RdBu_r)
    #colorbar(P4, ax=ax[1,1], shrink=0.8)

In [ ]:
for k in range(1):
    fig,ax=subplots(2,2)
    fig.set_size_inches((14,10))
    I = mcpol_c2['I_up (TOA)'][:].reshape((NX,NY))
    Q = mcpol_c2['Q_up (TOA)'][:].reshape((NX,NY))
    U = mcpol_c2['U_up (TOA)'][:].reshape((NX,NY))
    V = mcpol_c2['V_up (TOA)'][:].reshape((NX,NY))
    P1=ax[0,0].imshow(I.T, vmin=0., vmax=0.6, cmap=cm.jet)
    colorbar(P1, ax=ax[0,0], shrink=0.8)
    P2=ax[0,1].imshow(Q.T, vmin=-0.1, vmax=0.1, cmap=cm.RdBu_r)
    colorbar(P2, ax=ax[0,1], shrink=0.8)
    P3=ax[1,0].imshow(-U.T, vmin=-0.0005, vmax=0.0005, cmap=cm.RdBu_r)
    colorbar(P3, ax=ax[1,0], shrink=0.8)
    P4=ax[1,1].imshow(V.T, vmin=-0.0001, vmax=0.0001, cmap=cm.RdBu_r)
    colorbar(P4, ax=ax[1,1], shrink=0.8)
    #P4=ax[1,1].imshow(np.sqrt(Q**2+U**2)/I*100, vmin=0, vmax=40, cmap=cm.RdBu_r)
    #colorbar(P4, ax=ax[1,1], shrink=0.8)

# LPS2019

In [ ]:
# Layering and angles
'''le   = {'th_deg':np.array([20,20,20,20,40,40,40,40,40], dtype=float), 
        'phi_deg': np.array([0,60,120,180,0,0,60,120,180], dtype=float), 'zip':True}'''
#SENTINEL_SOLAR = REPTRAN('reptran_solar_sentinel')
wl_ref = 865.
pfwav= [wl_ref]
znew = np.array([100, 75, 50, 30, 20, 10, 5, 2., 0])
aer1 = AeroOPAC('continental_polluted', 1., wl_ref)
#ph=AtmAFGL('afglmw', grid=znew, OPT3D=True, comp=[aer1], pfwav= pfwav).calc(wl_ref, phase=True)['phase_atm']
cld  = CloudOPAC('wc.sol', 1., 8., 10., .5, wl_ref)

wl = np.linspace(680, 450, num=201)
#wl = [680., 550., 450.]
#wl = [865., 680., 450.]
#ibands = SENTINEL_SOLAR.to_smartg(include='sentinel2a_msi_b0', 
 #                           lmin=[400., 500., 600], lmax=[500., 600., 700.])
#wl = [wl_ref]
NL   = len(znew)
print('NLayer:', NL)
pfwav = wl_ref
   
pro_clr3D = AtmAFGL('afglmw', grid=znew, OPT3D=True,
                    pfwav= pfwav).calc(wl, phase=True)
pro1_cld3D = AtmAFGL('afglmw', grid=znew, comp=[cld], OPT3D=True,
                    pfwav=pfwav).calc(wl, phase=True)

In [ ]:
HLONGX = 1e4
HLONGY = 1e4
HX     = 4.
HY     = 4.
N  = 3
M  = 3
#N  = 1
#M  = 1
DX = 2*HX/N
DY = 2*HY/M
x  = np.zeros((N+2))
y  = np.zeros((M+2))
DXbound = (HLONGX-HX)
DYbound = (HLONGY-HY)
x[0]    = -HLONGX + DXbound/2
x[-1]   =  HLONGX - DXbound/2.
x[1:-1] =  np.linspace(-HX + DX/2., HX - DX/2., num=N)
y[0]    = -HLONGY + DYbound/2
y[-1]   =  HLONGY - DYbound/2.
y[1:-1] =  np.linspace(-HY + DY/2., HY - DY/2., num=M)
X0, Y0  =  np.meshgrid(x,y)
Lx=[DXbound]
Lx.extend([DX]*N)
Lx.extend([DXbound])
Ly=[DYbound]
Ly.extend([DY]*M)
Ly.extend([DYbound])
DX0, DY0=  np.meshgrid(Lx,Ly)
H = []
for xi,yi,dxi,dyi in zip(X0.ravel(),Y0.ravel(),DX0.ravel(),DY0.ravel()):
    H.append(list((xi-dxi/2, xi+dxi/2, yi-dyi/2, yi+dyi/2)))
H_boundaries = np.array(H).reshape(((N+2)*(M+2),4))
# 1x1
'''plist=[pro_clr3D]*4
plist.extend([pro_cld3D])
plist.extend([pro_clr3D]*4)'''

# 2x2 (avec 2 clouds differents)
'''plist=[pro_clr3D]*5
plist.extend([pro1_cld3D, pro2_cld3D])
plist.extend([pro_clr3D]*2)
plist.extend([pro2_cld3D, pro1_cld3D])
plist.extend([pro_clr3D]*5)'''

# 3x3 (anneau)
plist=[pro_clr3D]*6
plist.extend([pro1_cld3D]*3)
plist.extend([pro_clr3D]*2)
plist.extend([pro1_cld3D, pro_clr3D, pro1_cld3D])
plist.extend([pro_clr3D]*2)
plist.extend([pro1_cld3D]*3)
plist.extend([pro_clr3D]*6)

p3D_NxM = pro3D_from_1D(plist, H_boundaries=H_boundaries).describe()

In [ ]:
%%time
POSZ = 19.9999
THDEG, PHDEG = 170., 0.
#pixels
NX = 40
NY = 40
NZ = 10
NP = NX*NY
VX = 10.
VY = 10.
VZ = 4.9999
Ix = np.linspace(-VX, VX, num=NX) 
Iy = np.linspace(-VY, VY, num=NY) 
Iz = np.linspace(0.0001, VZ, num=NZ) 
px, py   = np.meshgrid(Ix, Iy)

poss = []
for POSX,POSY in zip(px.ravel(), py.ravel()):
    poss.append(Point(POSX, POSY, POSZ))
iboxes = get_iboxes(p3D_NxM, poss)

sensors=[]
for pos,IBOX in zip(poss,iboxes):
    sensors.append(Sensor(POSZ=pos.z, POSX=pos.x, POSY=pos.y, FOV=90., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX))
    

IBOX0   = get_iboxes(p3D_NxM, [Point(0, 0, POSZ)])
sensor0 = Sensor(POSZ=POSZ, THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', IBOX=IBOX0)   

In [ ]:
import ephem

def Get_SunEarth(obs):
    """
    Get Earth-Sun distance correction
    """
    date = str(obs.date).split()[0]
    jday = datetime.datetime.strptime(date, '%Y/%m/%d').timetuple().tm_yday
    SE_corr_c1 = 1.00011
    SE_corr_c2 = 0.034221
    SE_corr_c3 = 0.00128
    SE_corr_c4 = 0.000719
    SE_corr_c5 = 0.000077

    d_qo = 2.0 * np.arccos(-1.) * (jday + 0.35) / 365.
    d_corr = SE_corr_c1 +                       \
             SE_corr_c2 * np.cos(d_qo) +           \
             SE_corr_c3 * np.sin(d_qo) +           \
             SE_corr_c4 * np.cos(d_qo * 2.) +      \
             SE_corr_c5 * np.sin(d_qo * 2.)
    dist = 1.0 / np.sqrt(d_corr)

    return dist * dist


lon = 0.
lat = 35.

dates= ['2015/6/21','2015/9/20']

dt  = 75./(24*60) # time interval in fraction of day, here 30 minutes

obs      = ephem.Observer()
obs.lon  = str(lon)
obs.lat  = str(lat)
obs.pressure=0.
obs.horizon = '0:34'
obs.date = dates[1]
sun      = ephem.Sun()
drise    = obs.next_rising (sun, start=obs.date)
dset     = obs.next_setting(sun, start=obs.date)
dnoon    = obs.next_transit(sun, start=obs.date)
#halfday  = dnoon - drise # computation from noon to sunrise
fullday  = dset  - drise # computation from noon to sunrise

#t        = np.linspace(0.001, halfday, num=halfday/dt, endpoint=True)
t        = np.linspace(0.00, fullday, num=fullday/dt, endpoint=False)

dates=[]
th0=[]
sed=[]
ph0 = []
for i,dt in enumerate(t):
    #da       = dnoon - dt
    da       = dset  - dt
    obs.date = da
    sun.compute(obs)
    th0.append( 90.-float(sun.alt)*180/np.pi)
    ph0.append(float(sun.az)*180./np.pi)
    dates.append(str(obs.date))
    sed.append(Get_SunEarth(obs))
th0=np.array(th0)
ph0=np.array(ph0)
mus = cos(th0*np.pi/180)

# Solar geometries
le={'th_deg':th0, 'phi_deg':ph0, 'zip':True}

In [ ]:
%%time
# case 5
water = IOP_1(10.1, ang_trunc=30, DEPTH=10, FQYC=0.1)
water = None
ALB = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
#ALB = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.dry.solid.drygras.spectrum.txt')
'''le   = {'th_deg':np.array([0], dtype=float), 
        'phi_deg': np.array([0], dtype=float), 'zip':True}'''
surf= RoughSurface()
#env = Environment(ENV=1, ALB=Albedo_cst(0.2), ENV_SIZE=3, X0=2.5)
env = Environment(ENV=1, ALB=ALB, ENV_SIZE=3, X0=2.5)
env = None
NB=1e4*NP
#NB=1e7
#m3D_NxM  = S3D.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensors, env=env, water=water,
                   #reflectance=True, le=le, surf=surf, stdev=True)
NB=1e6
m3D_NxM0  = S3D.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensor0, env=env, water=water,
                   reflectance=True, le=le, surf=surf, stdev=True, alis_options={'nlow':-1})
#m3D_NxM  = S3D.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensors, env=env,
                   #reflectance=True, le=le, surf=surf, stdev=True)
#m3DA_NxM  = S3DA.run(wl=wl, NF=1e3, NBPHOTONS=NB, atm=p3D_NxM, sensor=sensors, env=env,
                   #reflectance=True, le=le, surf=surf, stdev=True, alis_options={'nlow':3})
print(m3D_NxM0.attrs['kernel time (s)'])

#smartg_c2 = m3D_NxM.dropaxis('Zenith angles').dropaxis('Azimuth angles').dropaxis('wavelength')

In [ ]:
m3D_NxM0.describe()

In [ ]:
from itertools import product
NROWS = 2
NCOLS = 4
f,ax = subplots(nrows=NROWS, ncols=NCOLS)
f.set_size_inches(12,6)
I = m3D_NxM['I_up (TOA)'].data
I2 = (I/I.max()*255).astype(int)
k=1
for i,j in product(np.arange(NROWS), np.arange(NCOLS)):
    ax[i,j].imshow(I2[:,:,k].reshape((NX,NY,3)))
    k+=1
f.savefig('/home/documents/TOSCA/Hagolle_3D.png', dpi=100)

In [ ]:
from itertools import product
NROWS = 2
NCOLS = 4
k=1
for i,j in product(np.arange(NROWS), np.arange(NCOLS)):
    m3D_NxM0['I_up (TOA)'].sub()[:,k].plot(vmin=0, vmax=.2)
    k+=1

In [ ]:
smartg_c2 = m3D_NxM.dropaxis('Zenith angles').dropaxis('Azimuth angles')

extent=[Ix[0], Ix[-1], Iy[0], Iy[-1]]
fig,ax=subplots(1,2)
fig.set_size_inches((14,10))
I = smartg_c2['I_up (TOA)'][:,:].reshape((NX,NY,3))
Q = smartg_c2['Q_up (TOA)'][:,:].reshape((NX,NY,3))
U = smartg_c2['U_up (TOA)'][:,:].reshape((NX,NY,3))
LP = np.sqrt(U**2+Q**2)
DoLP = LP/I*100
P1=ax[0].imshow(I, vmin=0.0, vmax=.3, cmap=cm.Blues_r, extent=extent)
colorbar(P1, ax=ax[0], shrink=0.5, label=r'$\rho^{TOA} (865 nm) $')
P4=ax[1].imshow(LP, vmin=0, vmax=.05, cmap=cm.Blues_r, extent=extent)
colorbar(P4, ax=ax[1], shrink=0.5, label='DoLP (%)')
ax[0].set_xlabel('X (km)')
ax[1].set_xlabel('X (km)')
ax[0].set_ylabel('Y (km)')
ax[1].set_ylabel('Y (km)')
fig.tight_layout()
fig.savefig('/home/documents/ESA/LPS2019/Lake_865.png', dpi=200)

fig,ax=subplots(1,2)
fig.set_size_inches((12,2))
ax[0].plot(Ix,I[int(NX/2),:,:],'+')
ax[0].set_xlabel('X (km)')
ax[0].set_ylim([0.005,0.25])
ax[0].set_title('Y=0 transect')
ax[0].set_ylabel(r'$\rho^{TOA} (865 nm) $')
ax[1].plot(Ix,LP[int(NX/2),:,:],'+')
ax[1].set_ylim([0.,0.25])
ax[1].set_ylabel('X (km)')
ax[1].set_title('Y=0 transect')
ax[1].set_ylabel('DoLP (%)')
fig.tight_layout()
fig.savefig('/home/documents/ESA/LPS2019/Lake_865_transect.png', dpi=200)

In [ ]:
extent=[Ix[0], Ix[-1], Iy[0], Iy[-1]]
fig,ax=subplots(1,2)
fig.set_size_inches((14,10))
I = smartg_c2['I_up (TOA)'][:].reshape((NX,NY))
Q = smartg_c2['Q_up (TOA)'][:].reshape((NX,NY))
U = smartg_c2['U_up (TOA)'][:].reshape((NX,NY))
LP = np.sqrt(U**2+Q**2)
DoLP = LP/I*100
P1=ax[0].imshow(I, vmin=0.0, vmax=.3, cmap=cm.Blues_r, extent=extent)
colorbar(P1, ax=ax[0], shrink=0.5, label=r'$\rho^{TOA} (865 nm) $')
P4=ax[1].imshow(DoLP, vmin=0, vmax=30, cmap=cm.Blues_r, extent=extent)
colorbar(P4, ax=ax[1], shrink=0.5, label='DoLP (%)')
ax[0].set_xlabel('X (km)')
ax[1].set_xlabel('X (km)')
ax[0].set_ylabel('Y (km)')
ax[1].set_ylabel('Y (km)')
fig.tight_layout()
fig.savefig('/home/documents/ESA/LPS2019/Lake_865.png', dpi=200)

fig,ax=subplots(1,2)
fig.set_size_inches((12,2))
ax[0].plot(Ix,I[int(NX/2),:],'+')
ax[0].set_xlabel('X (km)')
ax[0].set_ylim([0.005,0.05])
ax[0].set_title('Y=0 transect')
ax[0].set_ylabel(r'$\rho^{TOA} (865 nm) $')
ax[1].plot(Ix,DoLP[int(NX/2),:],'+')
ax[1].set_ylim([0.,40])
ax[1].set_ylabel('X (km)')
ax[1].set_title('Y=0 transect')
ax[1].set_ylabel('DoLP (%)')
fig.tight_layout()
fig.savefig('/home/documents/ESA/LPS2019/Lake_865_transect.png', dpi=200)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()

# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
for th in m3D_NxM.axis('Zenith angles'):
    I = m3D_NxM.sub({'Zenith angles':Idx(th)})['I_up (TOA)'][:,:].reshape((NX,NY,3))
    im=plt.imshow((I/I.max()*255).astype(int), animated=True)
    ims.append([im])
'''for i in range(60):
    x += np.pi / 15.
    y += np.pi / 20.
    im = plt.imshow(f(x, y), animated=True)
    ims.append([im])'''

#ani = animation.ArtistAnimation(fig, ims, interval=2000, blit=True,
 #                               repeat_delay=None)

# To save the animation, use e.g.
#
# ani.save("/home/did/work/movie.mp4")
#
# or
#
from matplotlib.animation import FFMpegWriter
writer = FFMpegWriter(fps=5, metadata=dict(artist='Me'), bitrate=1800)
ani.save("/home/did/work/movie.mp4", writer=writer)

plt.show()


# Profile to Cell

## Simple atmosphere

In [ ]:
Nx = 1
Ny = 1
Nz = 53
Dx = 1
Dy = 1
Dz = 1

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Nz=Nz, Dx=Dx, Dy=Dy, Dz=Dz, SAT_ALTITUDE=120, HORIZ_EXTENT_LENGTH=1e5, periodic=True)

# cells optical properties
iopt           = np.zeros(NX*NY*NZ, dtype= int32)
# properties depend only on z
iatm           = np.arange(NZ)
iopt[:]        = iatm[idz]+1

atm3D_clr = AtmAFGL('afglt', O3=0., NO2=False,
                    grid        = zgrid,
                    cells       = (iopt, pmin, pmax, neigh)
                   )
atm_clr   = AtmAFGL('afglt', O3=0., NO2=False,
                    grid        = zgrid[::-1]
                   )
wl = np.linspace(400., 420., num=5)
wl = [400.]

pro3D_clr = atm3D_clr.calc(wl)
pro_clr   = atm_clr.calc(wl)

In [ ]:
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax

In [ ]:
# Sensors position
POSX = 0.005
POSY = 0.005
POSZ = 110.00001
P    = Point(z=POSZ)
icells = locate_3D_cells(pro3D_clr, np.array([POSX]), np.array([POSY]), np.array([POSZ]))
ICELL = icells[0]
print(ICELL)
# pixels centers on the ground
x0    = xgrid # central domain (cumulus) boundaries
y0    = ygrid
# centers
xx,yy = np.meshgrid(np.diff(x0)/2. + x0[:-1], np.diff(y0)/2. + y0[:-1])
# view angles
PHDEG = np.degrees(np.arctan2(xx, yy))
THDEG = 180-np.degrees(np.arctan2(np.sqrt(xx**2+yy**2), POSZ))

sensors=[]
for th, ph in zip(THDEG.ravel(), PHDEG.ravel()):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=th, PHDEG=ph, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
S3D    = Smartg(opt3D=True,  alt_pp=True) # 3D
S      = Smartg(opt3D=False, alt_pp=True) # 3D
le     = {'th_deg':np.linspace(0, 80., num=1), 'phi_deg':np.linspace(0., 210., num=1)}

NB     = 1e7
surf   = RoughSurface()
surf   = None
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_clr, sensor=sensors, le=None, surf=surf, NBLOOP=1e6,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35)
m0     = S.run(wl=wl,  NBPHOTONS=NB, atm=pro_clr, sensor=sensors, le=None, surf=surf, NBLOOP=1e6,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35)

In [ ]:
_=smartg_view(m3D.sub({'sensor index':0}))
_=smartg_view(m0.sub({'sensor index':0}))

## Cloud

In [ ]:
# Cloud Phase matrix
cld_phase = read_mlut('/home/did/RTC/SMART-G/smartg/validation/IPRT/watercloud.mie.nc')
ntheta = cld_phase['ntheta'][0,0,0]
wl     = [670.]
l      = []
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() + cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,0,:ntheta].ravel() - cld_phase['phase'][:,:,1,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,2,:ntheta].ravel())
l.append(cld_phase['phase'][:,:,3,:ntheta].ravel())
# Normalization to 2
p      = cld_phase['phase'][:,:,0,:ntheta].ravel()
theta  = cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]
mu     = np.cos(np.radians(theta))
NORM   = np.trapz(p,-mu)
data   = np.array(l) 
print ('phase function norm: ', NORM)
#data = np.array(l) * (2./Norm)
# 2) store data in a LUT object with 4 dimensions, the z dimension is restricted to one level of altitude 0
pha_cld = LUT(data[:, ::-1], names=['stk', 'theta_atm'],
          axes=[None, cld_phase['theta'][:,:,0,:ntheta].ravel()[::-1]]
         )

### Step cloud C1

In [ ]:
#alternative reff = 2 micron phase function
wl = 1020.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

In [ ]:
Nx = 2
Ny = 1
Nz = 1
Dx = 0.5/Nx
Dy = 1e5
Dz = 0.25

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Nz=Nz, Dx=Dx, Dy=Dy, Dz=Dz, periodic=True,
                SAT_ALTITUDE=120, HORIZ_EXTENT_LENGTH=0)

Ncell   = NX*NY*NZ
Natm    = 3 # 2 cloud cells + void
# cells optical properties indices
iopt    = np.array([0, 1, 2, 1], dtype=int32)
# optical properties 
ptc_sca = np.array([2./0.25, 0., 18./0.25])
ptc_ssa = np.array([0.99, 1., 0.99])
ray_sca = np.zeros_like(ptc_sca)
gas_abs = np.zeros_like(ptc_sca)
ipha    = np.zeros_like(ptc_sca, dtype=int32)

In [ ]:
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax

In [ ]:
atm3D_cld = AtmAFGL('afglt',
                    grid        = zgrid,
                    prof_ray    = ray_sca[None,:],
                    prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )
wl = [800.]

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
# Sun position
SAA    = 180.
SZA    = 20.
# Sensors position
Npix   = 32
POSZ   = 0.25
THDEG  = 120.
PHDEG  = 135.
# pixels centers on the ground
x0    = np.linspace(xgrid[0]+Dx/2., xgrid[-1]-Dx/2., num=Npix) # central domain (cumulus) boundaries
y0    = np.array([0.])
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3D_cells(pro3D_cld, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D

In [ ]:
%%time
#S3D    = Smartg(opt3D=True,  alt_pp=True) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}

NB     = len(sensors) * 1e5
surf   = None
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35, alis_options={'nlow':-1})

In [ ]:
(m3D['I_up (TOA)'].sub()[:,0,0]/np.pi*1e3).plot(vmin=0, fmt='-')

In [ ]:
(m3D['V_up (TOA)'].sub()[:,0,0]/np.pi*1e3).plot( fmt='-')

In [ ]:
len(sensors)

### Cubic cloud C2

In [ ]:
#alternative reff = 2 micron phase function
wl = 1020.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

In [ ]:
#xgrid = np.array([-1e5, -3.5, -0.5, 0.5, 3.5, 1e5])
#ygrid = np.array([-1e5, -3.5, -0.5, 0.5, 3.5, 1e5])
#xgrid = np.array([ -3.5, -0.5, 0.5, 3.5])
#ygrid = np.array([ -3.5, -0.5, 0.5, 3.5])
Nx=1
Ny=1
Dx=1.
Dy=1.
zgrid = np.array([0., 2., 3., 5., 120.])

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)

Ncell   = NX*NY*NZ
#Natm    = 2 # 1 cloud cell + void
# cells optical properties indices
cloud_cell = np.ravel_multi_index((1, 1, 1), dims=(NX,NY,NZ), order='C')
iopt    = np.zeros(Ncell, dtype=int32)
iopt[cloud_cell] = 1
# optical properties 
ptc_sca = np.array([0., 10.])
ptc_ssa = np.array([1., 1.])
ray_sca = np.zeros_like(ptc_sca)
gas_abs = np.zeros_like(ptc_sca)
ipha    = np.zeros_like(ptc_sca, dtype=int32)

In [ ]:
atm3D_cld = AtmAFGL('afglt',
                    grid        = np.array([120.,0]),
                    #grid        = zgrid,
                    prof_ray    = ray_sca[None,:],
                    prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )
wl = [800.]

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
# Sun position
SAA    = 0.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 5.0001
POSX   = 3.4999
POSY   = 3.4999
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
#x0    = np.linspace(xgrid[1], xgrid[-2], num=Npix) # central domain (cumulus) boundaries
#y0    = np.linspace(ygrid[1], ygrid[-2], num=Npix)
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ

#xx,zz = np.meshgrid(x0, z0)
#yy    = np.zeros_like(xx) + POSY
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
%%time
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}

NB     = len(sensors) * 1e4
surf   = LambSurface(ALB=0.2)
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4, NBPHI=18, NBTHETA=35, alis_options={'nlow':-1})

In [ ]:
print(NB)
imshow(m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:], cmap='jet')
colorbar()
figure()
imshow(m3D['Q_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:], cmap='jet')
colorbar()

### Cumulus C3

In [ ]:
#alternative reff = 2 micron phase function
wl = 1020.
cld = CloudOPAC('wc.sol', 2., 2, 3., 1., wl)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl)['phase_atm'].sub()[0,:,:]

In [ ]:
# C3 IPRT Cumulus data
cumulus_f='/home/did/RTC/SMART-G/smartg/validation/IPRT/cumulus.dat'
f       = open(cumulus_f, 'r')
f.readline(); 
l       = f.readline().split()
Nx, Ny, Nz, _ = list(map(int, l)) # number of cumulus cells in X, Y, X
Nl      = Nz + 1                  # number of levels in Z
l       = f.readline()
g_info  = list(map(float, l.split()))
Dx, Dy  = g_info[0:2]              # horizontal grid interval in km
#zgrid   = np.array(g_info[2:][::-1])# vertical grid in km
zgrid   = np.array(g_info[2:])# vertical grid in km
assert Nl==zgrid.size

zgrid = np.concatenate([zgrid, np.array([120.])])
(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx, Ny=Ny, Dx=Dx, Dy=Dy, z=zgrid, SAT_ALTITUDE=120, periodic=False, HORIZ_EXTENT_LENGTH=1e5)
Ncell   = NX*NY*NZ

In [ ]:
(idx,idy,idz), (NX,NY,NZ), neigh, pmin, pmax

In [ ]:
# cumulus geometrical and optical data
cumulus = np.loadtxt(cumulus_f, skiprows=3, dtype=np.float32)
ptc_sca = cumulus[:,3]            # km-1
# we add a zero optical property for void
ptc_sca = np.concatenate([np.array([0.]), ptc_sca])
ptc_ssa = np.ones_like( ptc_sca)
ray_sca = np.zeros_like(ptc_sca)  # No Rayleigh
gas_abs = np.zeros_like(ptc_sca)  # No absorption
ipha    = np.zeros_like(ptc_sca, dtype=int32)

# indexing of C3 cells into SMART-G grid
cum_geo = cumulus[:,:3].astype(int32)-1
cum_cell= np.ravel_multi_index((cum_geo[:,0], cum_geo[:,1], cum_geo[:,2]),
                    dims=(NX, NY, NZ))
# cells optical properties
iopt    = np.zeros(Ncell, dtype=int32)
iopt[cum_cell] = np.arange(len(cum_cell)) + 1 # number zero is reserved for void

In [ ]:
atm3D_cld = AtmAFGL('afglt',
                    grid        = zgrid,
                    prof_ray    = ray_sca[None,:],
                    prof_abs    = gas_abs[None,:],
                    prof_aer    = (ptc_sca[None,:], ptc_ssa[None,:]),
                    prof_phases = (ipha[None,:], [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D_cld = atm3D_cld.calc(wl).describe()

In [ ]:
# Sun position
SAA    = 180.
SZA    = 40.
# Sensors position
Npix   = 100
POSZ   = 30
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
x0     = np.linspace(-6.67/2., 6.67/2, num=Npix) # central domain (cumulus) boundaries
y0     = np.linspace(-6.67/2., 6.67/2, num=Npix)
#
xx,yy  = np.meshgrid(x0, y0)
zz     = np.zeros_like(xx) + POSZ

In [ ]:
%%time
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid,zgrid, xx.ravel(), yy.ravel(), zz.ravel())
#icells_old = locate_3D_cells(pro3D_cld, xx.ravel(), yy.ravel(), zz.ravel())
sensors=[]
for POSX,POSY,ICELL in zip(xx.ravel(), yy.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))

In [ ]:
S3D    = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}
#ALB    = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.grass.green.solid.gras.spectrum.txt')
#env    = Environment(ENV=1, ALB=ALB, ENV_SIZE=2, X0=-1.5)
env    = None

In [ ]:
%%time
NB     = len(sensors) * 1e3
#surf   = RoughSurface(WIND=5)
surf   = LambSurface(ALB=0.2)
m3D    = S3D.run(wl=wl,  NBPHOTONS=NB, atm=pro3D_cld, sensor=sensors, le=le, surf=surf, NBLOOP=1e6, env=env,
               stdev=False, NF=1e4, alis_options={'nlow':-1})

In [ ]:
figure(figsize=(6,6))
print(NB)
imshow(m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:],vmax=1, cmap='jet')
colorbar()
figure(figsize=(6,6))
I=m3D['I_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
Q=m3D['Q_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
U=m3D['U_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
V=m3D['V_up (TOA)'].sub()[:,0,0].data.reshape(Npix,Npix)[::-1,:]
imshow(np.sqrt(Q*Q+U*U+V*V)/I*100, vmax=5,cmap='jet')
colorbar()

## ALIS

In [ ]:
import hapi
import scipy.constants as cst
from Voigt_gpu import absorptionCoefficient_Voigt_gpu
from smartg.rrs import *
#Solar Spectrum raw data
#datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_full.dat')
datas = np.loadtxt('/home/applis/libRadtran/libRadtran-2.0.2/data/solar_flux/kurudz_0.1nm.dat')

In [ ]:
########## wavelength range target # nm
#lmin= 759.001
#lmax= 770.
lmin= 686.001
lmax= 698.
# Solar spectrum data
wl0 = datas[::1,0]
# from mW/m2/nm to photons/cm2/s/nm
E0  = datas[::1,1] * 1e-3 * 1e-4  / (cst.h *cst.c)  * (wl0*1e-9)
# Solar grid
#ii = np.where(( wl0>=lmin) & (wl0 <=lmax))
#wl = wl0[ii]
#NW = wl.size
# spectral resolution for absorption features # nm 
dl  = .005
NW  = int((lmax-lmin)/dl) + 1
wl  = np.linspace(lmin, lmax, num=NW) # wavelength grid

## RRS excitation wavelength grid for 273°K
wl_RRS, _ = L2d_inv(wl, 90., 273.)
lmin_RRS = wl_RRS.min()
lmax_RRS = wl_RRS.max()

# Solar spectrum LUT building
ii = np.where(( wl0>lmin_RRS) & (wl0 <lmax_RRS))
Es_LUT = LUT(E0[ii], axes=[wl0[ii]], names=['wavelength'], desc='Es')

# approximate spectral resolution for scattering computations # nm 
dls = 5.
NWS = int((lmax_RRS - lmin_RRS)/dls) 
NWS = NWS if is_odd(NWS) else NWS+1
wls = np.linspace(lmin_RRS, lmax_RRS, num=NWS)# scattering wavelength grid
#wls = np.linspace(400, lmax_RRS, num=NWS)# scattering wavelength grid
# number of aerosols phase function computed in the spectral interval
NWP = 1
pfwav= np.linspace(lmin_RRS, lmax_RRS, num=NWP)# phase functions wavelength grid

pnew = np.array([1000,975,950,925,900,875,850,825,800,775,\
            750,700,650,600,550,500,450,400,350,300,\
            250,225,200,175,150,125,100,70,50,30,\
            20,10,7,5,3,2,1], dtype=np.float32)
#
#znew = ( -np.log(pnew/pnew[0])*8)[::-1]
znew    = np.array([100., 99., 20., 10., 5., 3., 2., 0.])
HTOA = znew[0]

########## Atmosphere
atm = AtmAFGL('afglus', grid=znew, O3=0., NO2=True, pfwav=pfwav)
NLE = atm.prof.z.size

## Compilation of smartg in ALIS mode++++++++++++++++++++++++++++------------------------------------------------
Sm   = Smartg(alis=True, pp=True,  alt_pp=True, device=0, bias=True)
Sref = Smartg(alis=False, pp=True, alt_pp=True, device=0, bias=True)

# Solar spectrum
Es = Es_LUT[Idx(wl)]
Es_RRS = Es_LUT[Idx(wl_RRS, fill_value='extrema')]

# Ring normalized spectrum (Wagner et al., 2009, AMT)
wl_RRS, LRRS = L2d_inv(wl, 90., 273.)
fnorm  = np.sum(Es_RRS * LRRS , axis=1) / Es - 1.

# parameters for 1D linear interpolation of w
f  =  interp1d(wls,np.linspace(0, NWS-1, num=NWS))
iw =  f(wl)
#iw =  f(wl_RRS.ravel())
iwls_in = np.floor(iw).astype(int8)        # index of lower wls value in the wls array, 
wwls_in = (iw-iwls_in).astype(np.float32)  # floating proportion between iwls and iwls+1
# special case for NWS
ii = np.where(iwls_in==(NWS-1))
iwls_in[ii] = NWS-2
wwls_in[ii] = 1.

#Connect to HITRAN database
hapi.db_begin('data')
DOWNLOAD = True # True if first run
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
if DOWNLOAD:
    hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
    hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
    hapi.fetch('O2i3',7,3,vmin-100,vmax+100)

# prepare array for o2 absorption coefficients
ao2=np.zeros((NW+5,NLE), dtype=np.float64)

# compute O2 absorption coefficient and wavelength grid  with 'GPU' version of HAPI Voigt profile function
j=0
for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):
    nuo2,coefo2 = absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=True,
        OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
        Environment={'p':p/1013.,'T':t})
    ao2[:nuo2.size,j] = coefo2 * o2 * 1e5
    j=j+1
wlabs=(1e7/nuo2)

# back to increasing wavelengths

wlabs = wlabs[::-1]
ao2   = ao2[:nuo2.size,:]
ao2   = ao2[::-1,:]

#interpolation into the solar grid
ab = LUT(ao2, axes=[wlabs, atm.prof.z], names=['wavelength', 'z'] )
ao2_int= ab[Idx(wl, fill_value='extrema'),:]

NW = ao2_int.shape[0]
sigma = ao2_int
print(NW)

In [ ]:
#alternative reff = 2 micron phase function
#znew    = np.array([100., 99., 20., 10., 5., 3., 2., 0.])
wl_ref  = 690.
#cld     = CloudOPAC('wc.sol', 2., znew[-2], znew[-3], 10., wl_ref)
cld     = CloudOPAC('wc.sol', 2., znew[-2], znew[-3], 10., wl_ref)
pha_cld = AtmAFGL('afglt', comp=[cld]).calc(wl_ref)['phase_atm'].sub()[0,:,:]

In [ ]:
Nx=1
Ny=1
Dx=1.
Dy=1.
#zgrid = np.array([0., 2., 3., 5., 20., 99., 100.])
zgrid = znew[::-1]

(idx,idy,idz), (NX,NY,NZ), (xgrid, ygrid, zgrid), neigh, pmin, pmax = \
    Get_3Dcells(Nx=Nx,Dx=Dx,Ny=Ny,Dy=Dy, z=zgrid, SAT_ALTITUDE=120, 
                periodic=False, HORIZ_EXTENT_LENGTH=1e5)

Ncell   = NX*NY*NZ
Nopt    = len(zgrid) + 1 # one clear atmosphere + 1 cloud cell 

# cells optical properties indices
idx_cloud = 1
idy_cloud = 1
idz_cloud = 1

cloud_cell = np.ravel_multi_index((idx_cloud, idy_cloud, idz_cloud), dims=(NX,NY,NZ), order='C')
iopt       = np.zeros(Ncell, dtype=int32)
iopt[:]    = np.arange(Nopt)[::-1][idz+1]
iopt[cloud_cell] = Nopt - 1

# optical properties 
atm1D_clr  = AtmAFGL('afglt', O3=0., NO2=False, grid=znew).calc(wls)
atm1D_cld  = AtmAFGL('afglt', O3=0., NO2=False, comp=[cld], grid=znew).calc(wls)
ptc_sca    = np.concatenate([od2k(atm1D_clr, 'OD_p')[:,:], od2k(atm1D_cld, 'OD_p') [:,-idz_cloud-1:-idz_cloud]], axis=1)
ray_sca    = np.concatenate([od2k(atm1D_clr, 'OD_r')[:,:], od2k(atm1D_cld, 'OD_r') [:,-idz_cloud-1:-idz_cloud]], axis=1)
gas_abs    = np.concatenate([od2k(atm1D_clr, 'OD_g')[:,:], od2k(atm1D_cld, 'OD_g') [:,-idz_cloud-1:-idz_cloud]], axis=1)

atm1D_clr0  = AtmAFGL('afglt', O3=0., NO2=False, grid=znew).calc(wls[0])
atm1D_cld0  = AtmAFGL('afglt', O3=0., NO2=False, comp=[cld], grid=znew).calc(wls[0])
ptc_sca0    = np.concatenate([od2k(atm1D_clr0, 'OD_p')[:,:], od2k(atm1D_cld0, 'OD_p') [:,-idz_cloud-1:-idz_cloud]], axis=1)
ray_sca0    = np.concatenate([od2k(atm1D_clr0, 'OD_r')[:,:], od2k(atm1D_cld0, 'OD_r') [:,-idz_cloud-1:-idz_cloud]], axis=1)
gas_abs0    = np.concatenate([od2k(atm1D_clr0, 'OD_g')[:,:], od2k(atm1D_cld0, 'OD_g') [:,-idz_cloud-1:-idz_cloud]], axis=1)

ptc_ssa0    = np.ones_like(ptc_sca0)
ipha0       = np.zeros_like(ptc_sca0, dtype=int32)
ptc_ssa    = np.ones_like(ptc_sca)
ipha       = np.zeros_like(ptc_sca, dtype=int32)

# parameters for 1D linear interpolation of w
f  =  interp1d(wls,np.linspace(0, NWS-1, num=NWS))
iw =  f(wl)
#iw =  f(wl_RRS.ravel())
iwls_in = np.floor(iw).astype(int8)        # index of lower wls value in the wls array, 
wwls_in = (iw-iwls_in).astype(np.float32)  # floating proportion between iwls and iwls+1
# special case for NWS
ii = np.where(iwls_in==(NWS-1))
iwls_in[ii] = NWS-2
wwls_in[ii] = 1.

In [ ]:
ray_sca[0,:], ptc_sca[0,:], gas_abs[0,:], atm1D_cld.axis('z_atm'), idz, idz_cloud, iopt

In [ ]:
iopt, pmin[2,cloud_cell], iopt[cloud_cell], ptc_sca[0,iopt[cloud_cell]], gas_abs[0,iopt[cloud_cell]],  ray_sca[0,iopt[cloud_cell]], Nopt

In [ ]:
ptc_sca[0,iopt[cloud_cell]]

In [ ]:
atm3D = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = ray_sca,
                    #prof_abs    = gas_abs,
                    prof_aer    = (ptc_sca, ptc_ssa),
                    prof_phases = (ipha, [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D = atm3D.calc(wls).describe()
atm3D0 = AtmAFGL('afglt',
                    grid        = np.arange(Nopt),
                    prof_ray    = ray_sca0,
                    #prof_abs    = gas_abs,
                    prof_aer    = (ptc_sca0, ptc_ssa0),
                    prof_phases = (ipha0, [pha_cld]),
                    cells       = (iopt, pmin, pmax, neigh)
                   )

pro3D0 = atm3D0.calc(wls[0]).describe()

# Sun position
SAA    = 0.
SZA    = 40.
# Sensors position
Npix   = 70
POSZ   = 49.0001
POSX   = 3.4999
POSY   = 3.4999
THDEG  = 180.
PHDEG  = 0.
# pixels centers on the ground
x0    = np.linspace(-POSX, POSX, num=Npix) # central domain (cumulus) boundaries
y0    = np.linspace(-POSY, POSY, num=Npix)
z0    = np.linspace( POSZ,   5 , num=Npix)
#
xx,yy = np.meshgrid(x0, y0)
zz    = np.zeros_like(xx) + POSZ
# cells indices
icells = locate_3Dregular_cells(xgrid, ygrid, zgrid, xx.ravel(), yy.ravel(), zz.ravel())

sensors=[]
for POSX,POSY,POSZ,ICELL in zip(xx.ravel(), yy.ravel(), zz.ravel(), icells):
    sensors.append(Sensor(POSX=POSX, POSY=POSY, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', ICELL=ICELL))
    
sensor_cld = Sensor(POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, 0., 0., POSZ))

sensor_grd = Sensor(POSX = -POSX, POSZ=POSZ, FOV=0., TYPE=0,
                          THDEG=THDEG, PHDEG=PHDEG, LOC='ATMOS', 
                    ICELL=locate_3Dregular_cells(xgrid, ygrid, zgrid, -POSX, 0., POSZ))

In [ ]:
%%time
ALB     = Albedo_speclib('/rfs/data/speclib2.0/data/jhu.becknic.vegetation.trees.deciduous.solid.decidou.spectrum.txt')
S3Da    = Smartg(opt3D=True,  alt_pp=True, alis=True) # 3D
S3D     = Smartg(opt3D=True,  alt_pp=True, alis=False) # 3D
le      = {'th_deg':np.array([SZA]), 'phi_deg':np.array([SAA])}

surf    = LambSurface(ALB=1.)
NB      = 1e4
m       = S3Da.run(wl=wls,  NBPHOTONS=NB, atm=pro3D, sensor=sensor_grd, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4, alis_options={'nlow':NWS, 'hist':True}).sub({'Azimuth angles':0, 'Zenith angles':0}).describe()

surf    = LambSurface(ALB=ALB.get(wls[0]))
NB      = len(sensors) * 1e4
m0      = S3D .run(wl=wls[0],  NBPHOTONS=NB, atm=pro3D0, sensor=sensors, le=le, surf=surf, NBLOOP=1e5,
               stdev=False, NF=1e4).sub({'Azimuth angles':0, 'Zenith angles':0}).describe()

In [ ]:
%%time
NL    = Nopt-1 # number of opt properties
w     = m['disth_up (TOA)'][:, NL+4:-3]
ngood = np.sum(w[:,0]!=0)

S       = np.zeros((ngood,4),dtype=np.float32) 
cd      = m['disth_up (TOA)'][:ngood,     :NL  ]
S[:,:4] = m['disth_up (TOA)'][:ngood, NL  :NL+4]
w       = m['disth_up (TOA)'][:ngood, NL+4:-3  ]
nrrs    = m['disth_up (TOA)'][:ngood,      -3  ]
nref    = m['disth_up (TOA)'][:ngood,      -2  ]
nsif    = m['disth_up (TOA)'][:ngood,      -1  ]

XBLOCK  = 512
XGRID   = 512
NT      = XBLOCK*XGRID       # Maximum Number of threads
NPHOTON = cd.shape[0]        # Number of photons

NLAYER  = sigma.shape[1]     # Number of vertical layer
NWVL    = sigma.shape[0]     # Number of wavelength for absorption and output
NGROUP  = NT//NWVL           # Number of groups of photons
NTHREAD = NGROUP*NWVL        # Number of threads used
NBUNCH  = NPHOTON//NGROUP    # Number of photons per group
NP_REST = NPHOTON%(NGROUP*NBUNCH) # Number of additional photons in the last group
NWVL_LOW = NWS

print(NT, NTHREAD, NGROUP, NPHOTON, NLAYER, NWVL, NBUNCH, NP_REST, NWVL_LOW)
sigma_ab_in  = np.zeros((NLAYER, NWVL), order='C', dtype=np.float32)
sigma_ab_in[:,:]  = sigma.swapaxes(0,1)

alb_in       = ALB.get(wl).reshape(NWVL, order='C').astype(np.float32) 

cd_in        = cd.reshape((NPHOTON, NLAYER), order='C').astype(np.float32)
S_in         = S.reshape((NPHOTON, 4),       order='C').astype(np.float32)
weight_in    = w.reshape((NPHOTON, NWVL_LOW),order='C').astype(np.float32)
nrrs_in      = nrrs.reshape(NPHOTON,order='C').astype(np.int8)
nsif_in      = nsif.reshape(NPHOTON,order='C').astype(np.int8)
nref_in      = nref.reshape(NPHOTON,order='C').astype(np.int8)

In [ ]:
%%time
res_out      = gpuzeros((4, NWVL),   dtype=np.float64)
res_sca      = gpuzeros((4, NWVL),   dtype=np.float64)
res_rrs      = gpuzeros((4, NWVL),   dtype=np.float64)
res_sif      = gpuzeros((4, NWVL),   dtype=np.float64)
reduce_absorption_gpu(np.int64(NPHOTON), np.int64(NLAYER), np.int64(NWVL), np.int64(NTHREAD), np.int64(NGROUP), np.int64(NBUNCH), 
                      np.int64(NP_REST), np.int64(NWVL_LOW), res_out, res_sca, res_rrs, res_sif, 
                      to_gpu(sigma_ab_in), to_gpu(alb_in), to_gpu(cd_in), to_gpu(S_in), to_gpu(weight_in), to_gpu(nrrs_in), 
                      to_gpu(nref_in), to_gpu(nsif_in), to_gpu(iwls_in), to_gpu(wwls_in), 
                      block=(XBLOCK,1,1),grid=(XGRID,1,1))
I1 = res_out.get()/m[0][0,0]
#norm1 = I1[0,0]/mref['I_up (TOA)'][0,0]
norm1 = 1
I1_LUT = LUT(I1[0,:]/norm1, axes=[wl], names=['wavelength'], desc=mdesc('I_up (TOA)'))

In [ ]:
figure(figsize=(24,4))
plot(wl,alb_in,'b')
I1_LUT.plot()

figure(figsize=(24,4))
P_Raman1 = LUT(np.squeeze((weight_in.T * nrrs_in * np.squeeze(S_in[:,0])).sum(axis=1)/(weight_in.T * np.squeeze(S_in[:,0])).sum(axis=1)), 
    axes=[wls], names=['wavelength'], desc=r'$P_{Raman}$')
P_Raman1.sub({'wavelength':Idx(wl)}).plot()
P_Raman2 = res_rrs.get()/res_out.get()
plot(wl, P_Raman2[0,:])
ylim(0,0.01)

In [ ]:
figure(figsize=(24,4))
plot(wl,alb_in,'b')
I1_LUT.plot()

figure(figsize=(24,4))
P_Raman1 = LUT(np.squeeze((weight_in.T * nrrs_in * np.squeeze(S_in[:,0])).sum(axis=1)/(weight_in.T * np.squeeze(S_in[:,0])).sum(axis=1)), 
    axes=[wls], names=['wavelength'], desc=r'$P_{Raman}$')
P_Raman1.sub({'wavelength':Idx(wl)}).plot()
P_Raman2 = res_rrs.get()/res_out.get()
plot(wl, P_Raman2[0,:])
ylim(0,0.01)

In [ ]:
figure(figsize=(24,4))
plot(wl,alb_in,'b')
I1_LUT.plot()
ylim(0,0.4)

figure(figsize=(24,4))
P_Raman1 = LUT(np.squeeze((weight_in.T * nrrs_in * np.squeeze(S_in[:,0])).sum(axis=1)/(weight_in.T * np.squeeze(S_in[:,0])).sum(axis=1)), 
    axes=[wls], names=['wavelength'], desc=r'$P_{Raman}$')
P_Raman1.sub({'wavelength':Idx(wl)}).plot()
P_Raman2 = res_rrs.get()/res_out.get()
plot(wl, P_Raman2[0,:])
ylim(0,0.01)

In [ ]:
print(NB)
im = m0['I_up (TOA)'].data.reshape(Npix,Npix,1)[::-1,:,-1]
imshow(im, vmin=0, vmax=0.6, cmap='jet')
colorbar()

In [ ]:
print(NB)
im = m0['I_up (TOA)'].data.reshape(Npix,Npix,1)[::-1,:,-1]
imshow(im, vmin=0, vmax=0.6, cmap='jet')
colorbar()

In [ ]:
im = m3D['I_up (TOA)'].data.reshape(Npix,Npix,21)
plot(wl,im[Npix//2,Npix//2,:]/im[Npix//2,Npix//2-1,:],'b')
figure()
plot(wl,im[Npix//2,Npix//2-1,:],'b:')
plot(wl,im[Npix//2,Npix//3,:],'r')
plot(wl,im[0,0,:],'g')
ylim(0,0.5)

In [ ]:
im = m3D['Q_up (TOA)'].sub()[:,:,0,0].data.reshape(Npix,Npix,21)
plot(wl,im[35,35,:],'b')
plot(wl,im[35,15,:],'r')
plot(wl,im[0,0,:],'g')
#ylim(0,0.5)